In [ ]:
import numpy as np
import sounddevice as sd
import soundfile as sf
import threading

In [ ]:
event = threading.Event()

try:
    song, sampling_rate = sf.read('./local/sample.wav', always_2d=True)
    current_frame = 0

    def callback(output, frames, time, status):
        global current_frame
        print(time, ' # frames:', frames, ' current:', current_frame)
        if status:
            print(status)
        chunksize = min(len(song) - current_frame, frames)
        output[:chunksize] = song[current_frame:current_frame + chunksize]
        if chunksize < frames:
            output[chunksize:] = 0
            raise sd.CallbackStop()
        current_frame += chunksize
    
    stream = sd.OutputStream(
        samplerate=sampling_rate,
        channels=song.shape[1],
        callback=callback,
        finished_callback=event.set
    )

    with stream:
        event.wait()

except Exception as e:
    print(e)

In [ ]:
channel_sounds = music.split_to_mono()
samples = [s.get_array_of_samples() for s in channel_sounds]

fp_arr = np.array(samples).T.astype(np.float32)
fp_arr /= np.iinfo(samples[0].typecode).max

print(fp_arr.shape)

In [ ]:
sd.play(fp_arr, music.frame_rate)

In [ ]:
sd.stop()